In [1]:
import pandas as pd
import re
from itertools import chain

In [2]:
dt = '20190130'
data_dir = '/home/admin-/Documents/dict_data/' + dt +'/'
user_dict_names = ['colour_user_dict_' + dt + '.txt', 'candidate_user_dict_final_' + dt + '.txt']
# user_dict_names = ['colour_user_dict_'+ dt +'.txt', 'freq_user_dict_'+dt+'.txt', 'candidate_user_dict_final_' + dt + '.txt']
# candidate_user_dict_final_20190130 .txt
# freq_user_dict_20190130.txt

In [7]:
# 필요한 사용자 사전을 모두 합침
def get_user_dicts(user_dict_names):
    df_users = pd.DataFrame()
    for user_dict in user_dict_names:
        file_path = data_dir + user_dict
        df_users = df_users.append(pd.read_csv(file_path, names=['word']))
    df_users = df_users.reset_index(drop=True)
    return df_users

In [8]:
df_users = get_user_dicts(user_dict_names)
type(df_users)

pandas.core.frame.DataFrame

In [9]:
# candidate_user_dict_final_20190130 
df_users.word[:10]

0         회색 회색 색
1       회색깔 회색 색깔
2      회색색상 회색 색상
3      회색칼라 회색 칼라
4      회색컬러 회색 컬러
5      그래이색 그래이 색
6    그래이색깔 그래이 색깔
7    그래이색상 그래이 색상
8    그래이칼라 그래이 칼라
9    그래이컬러 그래이 컬러
Name: word, dtype: object

In [10]:
df_users.word = df_users.word.astype('str')
df_users.word = df_users.word.apply(lambda x: x.lower())
df_users = df_users.drop_duplicates().sort_values(by=['word'])
df_users = df_users.reset_index(drop=True)
# sr_user_dicts = df_user_dicts.line

In [12]:
# df_users
df_synonyms = pd.read_csv(data_dir+'synonyms_dict_'+dt+'.txt', sep=' => ', names=['synonyms', 'word'], doublequote=False)
df_synonyms.synonyms = df_synonyms.synonyms.apply(lambda x: re.sub('\"', '', x))
df_synonyms.word = df_synonyms.word.apply(lambda x: re.sub('\"', '', x))
df = df_synonyms.append(df_users, ignore_index=True).reset_index(drop=True)

/home/admin-/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/home/admin-/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [13]:
df.word[:10]


0         이중
1        사계절
2         체리
3         프릴
4         조이
5         일본
6        에이지
7    에이지투웨니스
8         에어
9         알리
Name: word, dtype: object

In [14]:
df.synonyms[:10]

0                                                   2중
1                                                  4계절
2                                           cherry, 채리
3                                   frilled, frill, 후릴
4                                              joy, 죠이
5                            japan, 재팬, 재펜, 저팬, 제팬, 제펜
6                                                  age
7    age20s, 에이지투웨니즈, 에이지트웨니스, 에이지투웨이스, 애이지투웨니스, 애이...
8                                                  air
9                                                  ali
Name: synonyms, dtype: object

In [15]:
df = df.sort_values(by=['word']).reset_index(drop=True)
df.synonyms = df.synonyms.astype('str')
df.synonyms = df.synonyms.fillna(value=0)
type(df.synonyms[1])

str

In [16]:
df['line'] = None


def word_and_synonyms(row):
    temp_str = row['word']
    if row['synonyms'] != 'nan':
        temp_list = [temp_str] + row['synonyms'].split(", ")
    else:
        temp_list = [temp_str]
    row['line'] = temp_list
    return row


df = df.apply(lambda row: word_and_synonyms(row), axis=1)

In [17]:
df.line.sample(20)

1360                                [초등학생, 초등생]
299                                        [들깨]
298                                 [드림, dream]
1069                                    [여성청결제]
1555                                    [트레이닝복]
50                                        [건강즙]
382                                       [만년필]
532                                  [반짝이, 빤짝이]
648                              [부라운칼라 부라운 칼라]
680                       [브랜드, brand, 브렌드, 상표]
827                              [소가죽, 쇠가죽, 우피]
1071    [여아, 여아동, 여자아이, 여자애, 여자어린이, 여쥬니어, 여주니어]
1366                         [초이스, choice, 쵸이스]
214                   [노랑, yellow, 노란, 옐로, 옐로우]
1398                               [카키색상 카키 색상]
278                                [도톰, 도툼, 두툼]
572                                  [버건디, 와인색]
806                                [세안, 세면, 세수]
1302                                     [중고도서]
1256                                      [전동차]
Name: line, dtype: object

In [18]:
user_words_list = df.line.tolist()
user_words_list = [_.lower() for _ in chain(*user_words_list)]
sr_user_dicts = pd.Series(user_words_list, name='word').sort_values().reset_index(drop=True)
sr_user_dicts = sr_user_dicts.astype('str')


In [19]:
len(sr_user_dicts)

3167

In [20]:
# user_dict 내
# ln 1:운동화
# ln 2:운동화 운동 신발 
# 같은 경우를 없앰

def drop_duplicates_data(sr_user_dicts):
    drop_index_list = []

    for i in range(len(sr_user_dicts)-1):
        if i == len(sr_user_dicts)-1: 
            break
        prev = sr_user_dicts[i].split(" ")
        nex = sr_user_dicts[i+1].split(" ")
        if prev[0] == nex[0]:
            drop_index_list.append(i)
    
    sr_user_dicts = sr_user_dicts.drop(drop_index_list).reset_index(drop=True)
    return sr_user_dicts

In [21]:
sr_user_dicts = drop_duplicates_data(sr_user_dicts)
len(sr_user_dicts)

3038

In [45]:
sr_user_dicts.sample(30)

1109          바이올렉
535             곶감
1920           야구공
1248            벨트
2564             크
620      남색깔 남색 색깔
803             등산
2941            헤어
2332            참깨
1260           보냉병
1472            상표
1393          블루레이
1688          스케처스
1835          쓰래기통
305          round
2025         오리털다운
2955           호박씨
2034           온도계
426            wow
2119           위생대
2838     풋살화 풋살 신발
654            냉풍기
1857           아동용
2394           츄니닝
1541         섬유유연재
47        buttoned
491     검정색깔 검정 색깔
3027       흰색 흰색 색
185            kit
346     smartphone
Name: word, dtype: object

In [34]:
len(user_words_list)

3038

In [37]:
prev_syns_list = []
new_user_words_list = []

user_words_list = sr_user_dicts.tolist()
user_words_list = [user_word.split() for user_word in user_words_list]
for user_word_list in user_words_list:
    word = user_word_list[0]
    new_line = word
    if len(user_word_list) == 1:
        pass
    else:
        if word != "".join(user_word_list[1:]):
            prev_syns_list.append(word + " => " + ", ".join(user_word_list[1:]))
        else:
            new_line = " ".join(user_word_list)
    new_user_words_list.append(new_line)

In [61]:
sr_prev_syns = pd.Series(prev_syns_list)

In [60]:
sr_user_dicts = pd.Series(new_user_words_list)

In [66]:
sr_user_dicts.to_csv(data_dir + 'user_dict_' + dt + '.txt', index=False)

In [67]:
sr_prev_syns.to_csv(data_dir + 'prev_synonyms_dict_' + dt + '.txt', index=False)